# Check that the Notebook Instance was Created Correctly

In [1]:
setup_instance_check_passed = False

In [2]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name='ec2', region_name=region)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Check Notebook Instance Name and Instance Type

In [3]:
import json
notebook_instance_name = None

try:
    with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
        data = json.load(notebook_info)
        resource_arn = data['ResourceArn']
        region = resource_arn.split(':')[3]
        notebook_instance_name = data['ResourceName']
    print('Notebook Instance Name: {}'.format(notebook_instance_name))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: COULD NOT RETRIEVE THE NOTEBOOK INSTANCE METADATA.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Notebook Instance Name: workshop


# Check Notebook Instance Settings

In [4]:
response = sm.describe_notebook_instance(
        NotebookInstanceName=notebook_instance_name
)
print(response)

{'NotebookInstanceArn': 'arn:aws:sagemaker:us-east-1:251166678609:notebook-instance/workshop', 'NotebookInstanceName': 'workshop', 'NotebookInstanceStatus': 'InService', 'Url': 'workshop.notebook.us-east-1.sagemaker.aws', 'InstanceType': 'ml.t3.medium', 'SubnetId': 'subnet-85bce3aa', 'SecurityGroups': ['sg-b56187c3'], 'RoleArn': 'arn:aws:iam::251166678609:role/TeamRole', 'NetworkInterfaceId': 'eni-06cf92e225b6ebb1c', 'LastModifiedTime': datetime.datetime(2020, 9, 24, 23, 38, 54, 488000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2020, 9, 24, 23, 36, 12, 446000, tzinfo=tzlocal()), 'DirectInternetAccess': 'Enabled', 'VolumeSizeInGB': 250, 'RootAccess': 'Enabled', 'ResponseMetadata': {'RequestId': '26e61ce8-37c3-43f4-9ebb-4b8b49f322da', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '26e61ce8-37c3-43f4-9ebb-4b8b49f322da', 'content-type': 'application/x-amz-json-1.1', 'content-length': '548', 'date': 'Fri, 25 Sep 2020 00:08:21 GMT'}, 'RetryAttempts': 0}}


In [5]:
notebook_instance_arn = None

try: 
    notebook_instance_arn = response['NotebookInstanceArn']
    print('[OK] Notebook Instance ARN: {}\n'.format(notebook_instance_arn))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: Could not find the notebook instance ARN in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Notebook Instance ARN: arn:aws:sagemaker:us-east-1:251166678609:notebook-instance/workshop



In [6]:
subnet_id = None

try: 
    subnet_id = response['SubnetId']
    print('[OK] Subnet ID: {}'.format(subnet_id))
except: 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: YOU NEED TO SET UP THE INSTANCE WITH VPC AND SUBNET. Could not find the SubnetId in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Subnet ID: subnet-85bce3aa


In [7]:
security_group = None

try: 
    security_group = response['SecurityGroups']
    print('[OK] Security Group: {}'.format(security_group))
except: 
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: YOU NEED TO SET UP THE INSTANCE WITH VPC AND SUBNET. Could not find SecurityGroups in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Security Group: ['sg-b56187c3']


In [8]:
direct_internet_access = None

try: 
    direct_internet_access = response['DirectInternetAccess']
    if direct_internet_access == 'Disabled':
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[ERROR]: YOU NEED TO SET UP THE INSTANCE WITH INTERNET ACCESS ENABLED')
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Direct Internet Access: {}'.format(direct_internet_access))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: CHECK YOU SET A VPC AND HAVE DIRECT INTERNET ACCESS ENABLED')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Direct Internet Access: Enabled


In [9]:
min_volume_size=250

In [10]:
volume_size = None

try:
    volume_size = response['VolumeSizeInGB']
    int(volume_size)
    if volume_size < min_volume_size: 
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[ERROR]: YOU ONLY HAVE {} GB VOLUME SIZE. YOU NEED TO SET UP YOUR INSTANCE WITH 250 GB VOLUME SIZE'.format(volume_size))
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Volume Size: {}'.format(volume_size))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: COULD NOT FIND THE VOLUME SIZE IN {}'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Volume Size: 250


# Summary: Check All Required Settings Are Set Correctly

In [11]:
if volume_size >= min_volume_size and direct_internet_access == 'Enabled' and security_group and subnet_id and notebook_instance_arn:
    setup_instance_check_passed=True
    print('[OK] Great Job!  Please Continue.')
else:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: WE HAVE IDENTIFIED A MISCONFIGURATION.')
    print('[ERROR]: STOP/DELETE THIS NOTEBOOK INSTANCE AND CREATE A NEW ONE FOLLOWING THE INSTRUCTIONS')
    print('[ERROR]: CURRENT (MISCONFIGURED) SETTINGS: volume_size = {}, direct_internet_access = {}, security_group = {}, subnet_id = {}, notebook_instance_arn = {}'.format(volume_size, direct_internet_access, security_group, subnet_id, notebook_instance_arn))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK] Great Job!  Please Continue.


# Please Make Sure You Don't See ^^ ERROR ^^ Message Above

# **_If You See Any ERROR Messages Above, You Must Do The Following:_** 

# **_1) Stop This Notebook Instance, 2) Wait For The Notebook Instance To Stop, 3) Delete This Notebook Instance, and 4) Re-Create This Notebook Instance._**

In [12]:
print(setup_instance_check_passed)

True


In [13]:
%store setup_instance_check_passed

Stored 'setup_instance_check_passed' (bool)


In [14]:
%store 

Stored variables and their in-db values:
setup_instance_check_passed             -> True


# Release Resources

In [15]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>